# The Sugar Trap: Market Gap Analysis

## Business Context
A global snack manufacturer plans to launch a healthy snacking product line.
While the market appears saturated with sugary snacks, the company lacks data-driven insight
into which product categories are under-served.

The objective of this analysis is to identify "Blue Ocean" opportunities in the snack aisle,
specifically categories where high-protein, low-sugar products are scarce.


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_csv(
    "en.openfoodfacts.org.products.csv",  sep="\t", 
    usecols=[
        "product_name",
        "categories_tags",
        "sugars_100g",
        "proteins_100g",
        "fat_100g",
        "ingredients_text"
    ],
    nrows=30000,
    low_memory=False
)


## Data Cleaning
The dataset contains missing values and unrealistic nutritional entries.
Products without valid sugar or protein data were removed to ensure reliable analysis.


df = df.dropna(subset=["product_name", "sugars_100g", "proteins_100g"])


df = df[
    (df["sugars_100g"].between(0, 100)) &
    (df["proteins_100g"].between(0, 80)) &
    (df["fat_100g"].between(0, 100))
]

df.shape


## Category Engineering
To simplify analysis, raw category tags were grouped into high-level snack categories
using keyword-based logic.


df["categories_tags"] = df["categories_tags"].fillna("").str.lower()

def assign_primary_category(tags):
    if "bar" in tags:
        return "Snack Bars"
    elif "chocolate" in tags:
        return "Chocolate & Candy"
    elif "biscuit" in tags or "cookie" in tags:
        return "Biscuits & Cookies"
    elif "nuts" in tags or "seeds" in tags:
        return "Nuts & Seeds"
    elif "protein" in tags:
        return "Protein Products"
    else:
        return "Other Snacks"

df["primary_category"] = df["categories_tags"].apply(assign_primary_category)

df["primary_category"].value_counts()


## Nutrient Analysis
Products were analyzed based on sugar and protein content to identify under-served
high-protein, low-sugar market segments.


HIGH_PROTEIN = 10
LOW_SUGAR = 5


plt.figure(figsize=(9,6))
plt.scatter(df["sugars_100g"], df["proteins_100g"], alpha=0.3)
plt.axvline(LOW_SUGAR)
plt.axhline(HIGH_PROTEIN)
plt.xlabel("Sugar (g per 100g)")
plt.ylabel("Protein (g per 100g)")
plt.title("Sugar vs Protein Across Snack Products")
plt.show()


gap_df = df[
    (df["proteins_100g"] >= HIGH_PROTEIN) &
    (df["sugars_100g"] <= LOW_SUGAR)
]

gap_df["primary_category"].value_counts()


category_summary = df.groupby("primary_category").agg(
    avg_sugar=("sugars_100g", "mean"),
    avg_protein=("proteins_100g", "mean"),
    product_count=("product_name", "count")
).reset_index()

category_summary.sort_values("product_count", ascending=False)


## Conclusion
The analysis reveals a clear gap in the snack market for high-protein, low-sugar products.
Snack Bars and Nuts-based categories show the strongest opportunity for innovation,
particularly products with at least 10g of protein and less than 5g of sugar per 100g.


In [ ]:
import pandas as pd
import os

# ----------------------------
# Step 1: Ensure your dataframe is ready
# ----------------------------
# Assuming your cleaned dataframe is named `df` from previous steps
df_subset = df.copy()

# Optional: limit to first 30,000 rows (you already do this, just in case)
df_subset = df_subset.head(30000)

# ----------------------------
# Step 2: Save CSV in current working directory
# ----------------------------
csv_filename = "products_sample.csv"
df_subset.to_csv(csv_filename, index=False)

print(f"✅ CSV saved as '{csv_filename}' in the current working directory:")
print(os.getcwd())

# ----------------------------
# Step 3: If using Google Colab, trigger download to local machine
# ----------------------------
try:
    from google.colab import files
    files.download(csv_filename)
    print("✅ CSV download triggered for your local machine.")
except ImportError:
    print("Not running in Colab — CSV is saved locally in the working directory.")
